# Generation-based MRC 문제를 풀어보기

### Requirements

In [1]:
# !pip install datasets==1.4.1
# !pip install transformers==4.4.1
# !pip install sentencepiece==0.1.95
# !pip install nltk

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 데이터 및 평가 지표 불러오기

In [3]:
from datasets import load_dataset

datasets = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [4]:
from datasets import load_metric

metric = load_metric('squad')

## Pre-trained 모델 및 토크나이저 불러오기

In [5]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)

In [6]:
model_name = "google/mt5-small"

In [7]:
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    config=config,
    cache_dir=None,
)

## 설정하기

In [8]:
max_source_length = 1024
max_target_length = 128
padding = False
preprocessing_num_workers=12
num_beams = 2
max_train_samples = 16
max_val_samples = 16
num_train_epochs = 3

## 전처리하기

In [9]:
def preprocess_function(examples):
    inputs = [f'question: {q}  context: {c} </s>' for q, c in zip(examples['question'], examples['context'])]
    targets = [f'{a["text"][0]} </s>' for a in examples['answers']]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["example_id"] = []
    for i in range(len(model_inputs["labels"])):
        model_inputs["example_id"].append(examples["id"][i])
    return model_inputs

In [10]:
column_names = datasets['train'].column_names

In [11]:
train_dataset = datasets["train"]
train_dataset[0]['answers']['text'][0]

'교향곡'

### 3개의 데이터로 함수 구조 확인

In [12]:
inputs = [f'question: {q}  context: {c} </s>' for q, c in zip(train_dataset['question'][:3], train_dataset['context'][:3])]
inputs[0]

'question: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?  context: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. </s>'

In [13]:
targets = [f'{a["text"][0]} </s>' for a in train_dataset['answers'][:3]]
targets[:3]

['교향곡 </s>', '1악장 </s>', '베토벤의 교향곡 9번 </s>']

In [14]:
model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

In [15]:
# model_inputs

In [16]:
with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets[:3], max_length=max_target_length, padding=padding, truncation=True)

model_inputs["labels"] = labels["input_ids"]
model_inputs["example_id"] = []
for i in range(len(model_inputs["labels"])):
    model_inputs["example_id"].append(train_dataset["id"][i])

In [17]:
model_inputs

{'input_ids': [[7680, 267, 5694, 11051, 10899, 988, 259, 83298, 22763, 649, 13502, 146064, 117921, 259, 47371, 1000, 7703, 43127, 611, 37494, 163646, 259, 232026, 929, 291, 19730, 267, 259, 89558, 2479, 5694, 11051, 10899, 988, 259, 83298, 22763, 649, 13502, 3216, 13042, 611, 36952, 6381, 259, 47371, 1000, 1884, 259, 29536, 873, 259, 29484, 747, 259, 68230, 4949, 259, 36709, 6345, 4843, 1235, 259, 16054, 15481, 649, 27512, 13237, 37398, 611, 37494, 159514, 259, 49082, 611, 259, 36114, 40329, 260, 1602, 259, 97030, 5694, 11051, 10899, 988, 259, 145086, 80135, 259, 59307, 27879, 69757, 2020, 259, 4793, 3227, 2277, 3227, 611, 3729, 259, 2, 869, 259, 34416, 50375, 259, 54796, 12929, 1603, 7801, 13614, 873, 4021, 40953, 138931, 23591, 12341, 132276, 217952, 77889, 1023, 12261, 26122, 13502, 3216, 13042, 649, 28463, 7108, 873, 5713, 10068, 82652, 21044, 260, 259, 7073, 839, 13502, 1751, 1613, 3759, 16072, 15961, 7145, 649, 5135, 78904, 1235, 13502, 1751, 83274, 3347, 13090, 10193, 37864, 500

In [18]:
train_dataset = datasets["train"]
train_dataset = train_dataset.select(range(max_train_samples))
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )

In [19]:
eval_dataset = datasets["validation"]
eval_dataset = eval_dataset.select(range(max_val_samples))
eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )


## Fine-tuning하기

In [20]:
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

In [21]:
label_pad_token_id = tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=label_pad_token_id,
            pad_to_multiple_of=None,
        )

In [22]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # decoded_labels is for rouge metric, not used for f1/em metric
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    formatted_predictions = [{"id": ex['id'], "prediction_text": decoded_preds[i]} for i, ex in enumerate(datasets["validation"].select(range(max_val_samples)))]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"].select(range(max_val_samples))]

    result = metric.compute(predictions=formatted_predictions, references=references)
    return result

In [23]:
args = Seq2SeqTrainingArguments(
    output_dir='outputs', 
    do_train=True, 
    do_eval=True, 
    predict_with_generate=True,
    num_train_epochs=num_train_epochs
)

In [24]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [25]:
train_result = trainer.train(resume_from_checkpoint=None)

Step,Training Loss


In [26]:
train_result

TrainOutput(global_step=6, training_loss=42.0323740641276, metrics={'train_runtime': 1.8691, 'train_samples_per_second': 3.21, 'total_flos': 43398356410368.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 58562, 'init_mem_gpu_alloc_delta': 1200707584, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 232167, 'train_mem_gpu_alloc_delta': 3616934400, 'train_mem_cpu_peaked_delta': 200823, 'train_mem_gpu_peaked_delta': 3617183232})

## 평가하기

In [27]:
metrics = trainer.evaluate(
    max_length=max_target_length, num_beams=num_beams, metric_key_prefix="eval"
)

In [28]:
metrics

{'eval_loss': 31.14606475830078,
 'eval_exact_match': 0.0,
 'eval_f1': 1.5625,
 'eval_runtime': 1.0018,
 'eval_samples_per_second': 15.971,
 'epoch': 3.0,
 'eval_mem_cpu_alloc_delta': 1955018,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 233827,
 'eval_mem_gpu_peaked_delta': 368118272}

In [29]:
document = "세종대왕은 언제 태어났어?"
input_ids = tokenizer(document, return_tensors='pt').input_ids
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0], skip_special_tokens=True)

RuntimeError: Input, output and indices must be on the current device